In [10]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")


/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 36873.00it/s]
/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [2]:
model.save_pretrained("../models/gliner_medium-v2.1")

In [12]:
model = GLiNER.from_pretrained("../models/gliner_medium-v2.1", load_onnx=True, load_tokenizer=True)

config.json not found in /workspaces/NER-project/models/gliner_medium-v2.1
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
ONNX_SAVE_PATH = "../models/gliner_medium-v2.1/model.onnx"

In [5]:
text = "ONNX is an open-source format designed to enable the interoperability of AI models across various frameworks and tools."
labels = ['format', 'model', 'tool', 'cat']

inputs, _ = model.prepare_model_inputs([text], labels)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
import torch

if model.config.span_mode == 'token_level':
    all_inputs =  (inputs['input_ids'], inputs['attention_mask'], 
                    inputs['words_mask'], inputs['text_lengths'])
    input_names = ['input_ids', 'attention_mask', 'words_mask', 'text_lengths']
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "words_mask": {0: "batch_size", 1: "sequence_length"},
        "text_lengths": {0: "batch_size", 1: "value"},
        "logits": {0: "position", 1: "batch_size", 2: "sequence_length", 3: "num_classes"},
    }
else:
    all_inputs =  (inputs['input_ids'], inputs['attention_mask'], 
                    inputs['words_mask'], inputs['text_lengths'],
                    inputs['span_idx'], inputs['span_mask'])
    input_names = ['input_ids', 'attention_mask', 'words_mask', 'text_lengths', 'span_idx', 'span_mask']
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "words_mask": {0: "batch_size", 1: "sequence_length"},
        "text_lengths": {0: "batch_size", 1: "value"},
        "span_idx": {0: "batch_size", 1: "num_spans", 2: "idx"},
        "span_mask": {0: "batch_size", 1: "num_spans"},
        "logits": {0: "batch_size", 1: "sequence_length", 2: "num_spans", 3: "num_classes"},
    }
print('Converting the model...')
torch.onnx.export(
    model.model,
    all_inputs,
    f=ONNX_SAVE_PATH,
    input_names=input_names,
    output_names=["logits"],
    dynamic_axes=dynamic_axes,
    opset_version=14,
)

Converting the model...


/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:547: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(mid - 1).type_as(relative_pos),
/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:551: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.ceil(torch.log(abs_pos / mid) / torch.log(torch.tensor((max_position - 1) / mid)) * (mid - 1)) 

In [7]:
# #quantize model
# import os
# from onnxruntime.quantization import quantize_dynamic, QuantType

# quantized_save_path = ("../models/gliner_medium-v2.1/model_quantized.onnx")
# # Quantize the ONNX model
# print("Quantizing the model...")
# quantize_dynamic(
#     ONNX_SAVE_PATH,  # Input model
#     quantized_save_path,  # Output model
#     weight_type=QuantType.QUInt8  # Quantize weights to 8-bit integers
# )

Quantizing the model...


: 

In [1]:
from gliner import GLiNER
model = GLiNER.from_pretrained("../models/gliner_medium-v2.1", load_onnx=True, load_tokenizer=True)

/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json not found in /workspaces/NER-project/models/gliner_medium-v2.1
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
text2 = "My name is Tom, I live in New York and my girlfriend's name is Elaine. Our parents live in Viet Nam, Nha Trang city, and their names are Que and Mai"
labels = ['Person', 'Place']

inputs, raw_batch = model.prepare_model_inputs([text2], labels)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
text = """
Libretto by Marius Petipa, based on the 1822 novella ``Trilby, ou Le Lutin d'Argail`` by Charles Nodier, first presented by the Ballet of the Moscow Imperial Bolshoi Theatre on January 25/February 6 (Julian/Gregorian calendar dates), 1870, in Moscow with Polina Karpakova as Trilby and Ludiia Geiten as Miranda and restaged by Petipa for the Imperial Ballet at the Imperial Bolshoi Kamenny Theatre on January 17–29, 1871 in St. Petersburg with Adèle Grantzow as Trilby and Lev Ivanov as Count Leopold.
"""

labels = ["person", "book", "location", "date", "actor", "character"]

entities = model.predict_entities(text, labels, threshold=0.4)

output_dict = {}

for entity in entities:
    output_dict[entity["text"]] = entity["label"]

for entity in entities:
    print(entity["text"], "=>", entity["label"])

Trilby => character
Moscow => location
January 25/February 6 => date
1870 => date
Moscow => location
Polina Karpakova => actor
Trilby => character
Ludiia Geiten => actor
Miranda => character
January 17–29, 1871 => date
St. Petersburg => location
Adèle Grantzow => actor
Trilby => character
Lev Ivanov => actor
Count Leopold => character


In [13]:
output_dict

{'Trilby': 'character',
 'Moscow': 'location',
 'January 25/February 6': 'date',
 '1870': 'date',
 'Polina Karpakova': 'actor',
 'Ludiia Geiten': 'actor',
 'Miranda': 'character',
 'January 17–29, 1871': 'date',
 'St. Petersburg': 'location',
 'Adèle Grantzow': 'actor',
 'Lev Ivanov': 'actor',
 'Count Leopold': 'character'}

In [3]:
import onnxruntime as ort
import numpy as np
ort_sess = ort.InferenceSession('../models/gliner_medium-v2.1/model.onnx')

In [4]:
import torch
outputs = ort_sess.run(None, {'input_ids': inputs['input_ids'].numpy(),
                            'attention_mask': inputs['attention_mask'].numpy(),
                            'words_mask': inputs['words_mask'].numpy(),
                            'text_lengths': inputs['text_lengths'].numpy(),
                            'span_idx': inputs['span_idx'].numpy(),
                            'span_mask': inputs['span_mask'].numpy(),
                            })[0]
outputs = torch.from_numpy(outputs)

In [5]:
outputs = model.decoder.decode(
            raw_batch["tokens"],
            raw_batch["id_to_classes"],
            outputs,
            flat_ner=True,
            threshold=0.5,
            multi_label=False,
        )[0]

outputs

[(3, 3, 'Person', 0.9672063589096069),
 (8, 9, 'Place', 0.8985015153884888),
 (17, 17, 'Person', 0.9670381546020508),
 (23, 24, 'Place', 0.9371719360351562),
 (26, 28, 'Place', 0.9003996849060059),
 (34, 34, 'Person', 0.8820420503616333),
 (36, 36, 'Person', 0.7397370934486389)]

In [6]:
texts = raw_batch['tokens'][0]

for output in outputs:
    start, end = output[:2]
    entity = output[2]
    print(f"{texts[start:end+1]} => {entity}")

['Tom'] => Person
['New', 'York'] => Place
['Elaine'] => Person
['Viet', 'Nam'] => Place
['Nha', 'Trang', 'city'] => Place
['Que'] => Person
['Mai'] => Person


In [9]:
import bentoml

text = """
Libretto by Marius Petipa, based on the 1822 novella ``Trilby, ou Le Lutin d'Argail`` by Charles Nodier, first presented by the Ballet of the Moscow Imperial Bolshoi Theatre on January 25/February 6 (Julian/Gregorian calendar dates), 1870, in Moscow with Polina Karpakova as Trilby and Ludiia Geiten as Miranda and restaged by Petipa for the Imperial Ballet at the Imperial Bolshoi Kamenny Theatre on January 17–29, 1871 in St. Petersburg with Adèle Grantzow as Trilby and Lev Ivanov as Count Leopold.
"""

labels = ["person", "book", "location", "date", "actor", "character"]
with bentoml.SyncHTTPClient('http://localhost:3000') as client:
    output_dict: dict = client.extract(text, labels)
    print(f"Result: {output_dict}")

Result: {'Trilby': 'character', 'Moscow': 'location', 'January 25/February 6': 'date', '1870': 'date', 'Polina Karpakova': 'actor', 'Ludiia Geiten': 'actor', 'Miranda': 'character', 'January 17–29, 1871': 'date', 'St. Petersburg': 'location', 'Adèle Grantzow': 'actor', 'Lev Ivanov': 'actor', 'Count Leopold': 'character'}
